In [ ]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

In [ ]:
# funtions 
# calculation of viscosity based on eq 22 from paper Sharqawy 2010 
def dynamic_viscosity_Sharqawy(t,S):
    muw = 4.2844e-5 + 1/ (0.157 * (t + 64.993)**2 - 91.296)
    A = 1.5413 + 1.998e-2 * t - 9.52e-5 * t**2
    B = 7.974 - 7.561e-2 *t + 4.724e-4 * t**2
    mu = muw * (1 + A * S + B * S**2)
    return mu

def buoyancy_drifter(
    diameter: float, heigth: float, mass: float, density_fluid: float
) -> float:
    volume_drifter = 1 / 4 * np.pi * diameter * diameter * heigth
    density_drifter = mass / volume_drifter
    buoyancy = density_drifter / density_fluid
    return buoyancy

def density_drifter(diameter: float, heigth: float, mass: float) -> float:
    volume_drifter = 1 / 4 * np.pi * diameter * diameter * heigth
    density_drifter = mass / volume_drifter
    return density_drifter

def stokes_relaxation_time(
    diameter: float, kinematic_viscosity: float, buoyancy: float
) -> float:
    """
    Calculates stokes relaxation time for MR equation
    - d: diameter [m]
    - kinematic_viscosity kinematic viscoscity sea water [m^2/s]
    - Buoyancy particle (rho_p / rho_w) [unitless]
    """
    return (1 + 2 * buoyancy) * diameter * diameter / (36 * kinematic_viscosity)

def Re_particle(Uslip: float, diameter: float, kinematic_viscosity: float) -> float:
    return np.abs(Uslip) * diameter / kinematic_viscosity

def factor_drag_white1991(Rep: float) -> float:
    """
    stokes drag corection factor valid up to Rep < 10^5 from
    [1] F. M. White, Viscous fluid flow 2nd edition (1991)(p. 182)
    dependencies:
    - Rep : particle reynolds kinematic_viscositymber
    """
    c_REp = 1 + Rep / (4.0 * (1 + np.sqrt(Rep))) + Rep / 60.0
    return c_REp

def Rep_geostrophic_uslip_stst_dim(B, d, f, nu, U,Rep):
    """
    calculates particle reynolds number based on geostrophic balance of
    steady state flow given by U=(U,0). 
    - B is buoyancy
    - d is particle diameter (assuming sphere)
    - f is coriolis paramter 
    - nu is kinematic viscosity
    - U is magnitude flow velocity
    """
    a = d/2
    R = 2 / (1+ 2 *B)
    crep = factor_drag_white1991(Rep)
    epsilon =   2 / 9 * (d/2 )**2 / (nu * crep)  * 1/R
    norm =((1/epsilon)**2 + f**2) 
    uslip = 2* (1-B) * f**2 * U / (   (1 + 2* B) *  norm ) 
    vslip = 18 * (1-B) * 1 * nu *  f * U / (  (1+2*B)**2 * a**2 *  norm ) 
    Uslip = np.sqrt(uslip**2+vslip**2)
    return Uslip*d/nu

def tau_drag(tau,Rep):
    """
    calculates drag time, i.e. timescale
    associated with drag force. Expression reduces
    to stokes relaxation time for Rep=0
    """
    c_rep = factor_drag_white1991(Rep)
    return tau/c_rep



In [ ]:
# properties drifters
d = 0.25 # m diameter outer ring
d_in = 0.2 # m (estimated) dimater inner ring, needed to calculate volume
h = 0.041  # m (heigth drifter)
m = 0.905 # kg (mass drifter)
omega_earth =  7.2921e-5 #[rad/sec]
f = 2 * omega_earth * np.sin(54/180 * np.pi) # coriolis at 54 deg N 

#properties water
av_temp_NWES = 11.983276
av_salinity_NWES = 34.70449
rho_water = 1027 # kg/m3 
dynamic_viscosity_water = dynamic_viscosity_Sharqawy(av_temp_NWES,av_salinity_NWES/1000)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water

# simulation values drifters
B = buoyancy_drifter(diameter  = d_in, heigth = h, mass = m, density_fluid = rho_water)
tau = stokes_relaxation_time(d, kinematic_viscosity_water, B)

For simulations with the MRG equations we need to set the stokes time and particle reynolds number (or keep is flexible)
Below I show how you can calculate the stokes relaxation time (tau) for a range of diamters and how you can estimate the particle reynolds number for a range of diameters (note that this is an estimate for which I asume a typical flow velocity of 1 m/s)

In [ ]:
# stokes time tau and estimate for particle reynolds number as function of diamter
dlist = np.arange(0.001,2,0.001)
taulist = stokes_relaxation_time(dlist, kinematic_viscosity_water, B)
Rep_geostrophic_estimate=[]
for d in dlist:
    def fie(Rep):
        return Rep_geostrophic_uslip_stst_dim(B,d,f,kinematic_viscosity_water,10,Rep)-Rep
    Rep_geostrophic_estimate.append(fsolve(fie,300)[0])
Rep_geostrophic_estimate = np.asarray(Rep_geostrophic_estimate)

In [ ]:
fig, ax = plt.subplots()
ax.plot(dlist,taulist,color='black')
ax2=ax.twinx()
ax2.plot(dlist,Rep_geostrophic_estimate,color='firebrick')
ax.set_xlabel('diameter [m]')
ax.set_ylabel('$\\tau_{\\mathrm{stokes}}$ [s]')
ax2.set_ylabel('Re$_p$',color='firebrick')

ax2.spines['right'].set_color('firebrick')
ax2.yaxis.label.set_color('firebrick')
ax2.tick_params(axis='y', colors='firebrick')

In [ ]:
fig, ax = plt.subplots() 
ax.plot(dlist,tau_drag(taulist,Rep_geostrophic_estimate))
ax.set_xlabel('diameter [m]')
ax.set_ylabel('$\\tau_{\\mathrm{drag}}$ [s]')

In [ ]:
#